# Synthetic Experiments with Multiple Sources in Each Batch

In [15]:
output_path = './outputs/graphs/'
results_path = './outputs/synthetic_results_batch_multiple_sources/'

In [16]:
import json
import os
import pandas as pd
import numpy as np

In [17]:
results_list = []
for file in [
    f for f in os.listdir(results_path) if f.endswith("json")
]:
    with open(results_path + file) as f:
        results_temp = json.load(f)
    results_list.append(results_temp)

In [18]:
results = {}

In [19]:
for results_dict in results_list:
    for dataset in results_dict:
        if dataset not in results:
            results[dataset] = {}
        for corruption_type in results_dict[dataset]:
            if corruption_type not in results[dataset]:
                results[dataset][corruption_type] = {}


            for run in results_dict[dataset][corruption_type]:
                if run not in results[dataset][corruption_type]:
                    results[dataset][corruption_type][run] = {}

                for depression in results_dict[dataset][corruption_type][run]:

                    results[dataset][corruption_type][run][depression] = [
                        dict(epoch=int(epoch), **metrics) 
                        for epoch, metrics in results_dict[dataset][corruption_type][run][depression].items() 
                        if epoch != 'corrupt_sources'
                    ]


In [20]:
results_df = []

for dataset in results:
    for corruption_type in results[dataset]:
        for run in results[dataset][corruption_type]:
            for depression in results[dataset][corruption_type][run]:
                results_df.append(
                    pd.json_normalize(
                        results[dataset][corruption_type][run][depression]
                    )
                    .assign(
                        dataset=dataset,
                        corruption_type=corruption_type,
                        run=run,
                        depression=depression,
                    )
                    .assign(epoch=lambda x: x['epoch']+1)
                )

results_df = pd.concat(results_df)
results_df = (
    results_df
    .replace({"depression": {"true": True, "false": False}})
    .astype(
        {
            "run": "int64",
            "depression": "bool",
        }
    )
)

In [7]:
with open(os.path.join(results_path, "baseline", "coteaching", "results.json")) as f:
    results_cot = json.load(f)


with open(os.path.join(results_path, "baseline", "idpa", "results.json")) as f:
    results_idpa = json.load(f)

In [8]:
results_cot_df = []

for dataset in results_cot:
    for corruption_type in results_cot[dataset]:
        for run in results_cot[dataset][corruption_type]:
            results_cot_df.append(
                pd.DataFrame(
                    results_cot[dataset][corruption_type][run]
                )
                .T
                .reset_index()
                .rename(columns={"index": "epoch"})
                .assign(
                    dataset=dataset,
                    corruption_type=corruption_type,
                    run=run,
                )
                .assign(epoch=lambda x: x['epoch'].astype(int)+1)
            )

results_cot_df = pd.concat(results_cot_df)
results_cot_df = (
    results_cot_df
    .astype(
        {
            "run": "int64",
        }
    )
)

In [9]:
results_idpa_df = []

for dataset in results_idpa:
    for corruption_type in results_idpa[dataset]:
        for run in results_idpa[dataset][corruption_type]:
            results_idpa_df.append(
                pd.DataFrame(
                    results_idpa[dataset][corruption_type][run]
                )
                .T
                .reset_index()
                .rename(columns={"index": "epoch"})
                .assign(
                    dataset=dataset,
                    corruption_type=corruption_type,
                    run=run,
                )
                .assign(epoch=lambda x: x['epoch'].astype(int)+1)
            )

results_idpa_df = pd.concat(results_idpa_df)
results_idpa_df = (
    results_idpa_df
    .astype(
        {
            "run": "int64",
        }
    )
)

In [10]:
results_cot_df = results_cot_df.assign(method="Co-teaching")
results_idpa_df = results_idpa_df.assign(method="IDPA")
results_df = results_df.assign(
    method=lambda df: df['depression'].map({True: "LAP", False: "Standard"})
)

In [11]:
columns_intersection = [
    'dataset', 'corruption_type', 'run', 'epoch', 
    'method', 'test_top1acc', 'test_top5acc'
]

results_all = (
    pd.concat([
        results_df[columns_intersection],
        results_cot_df[columns_intersection],
        results_idpa_df[columns_intersection],
    ])
    .assign(test_top1acc=lambda x: x['test_top1acc']*100)
    .groupby(['dataset', 'corruption_type', 'run', 'method'])
    .agg({'test_top1acc': 'max', 'test_top5acc': 'max'})
    .unstack()
    .swaplevel(axis=1)   
    .reset_index()
    .drop(columns='run')
    .groupby(['dataset', 'corruption_type'])
    .agg(['mean', 'std'])
    .stack(0)
    .stack(0)
    .assign(
        mean_std = lambda x: 
            np.round(x['mean'], 2).astype(str) 
            + " ± "
            + np.round(x['std'], 2).astype(str),
    )
    .drop(columns=['mean', 'std'])
    .unstack()
    .unstack()
)

/var/folders/dn/w2011ntd5gxf26dgf_38535m0000gp/T/ipykernel_23542/3814816774.py:18: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns='run')
/var/folders/dn/w2011ntd5gxf26dgf_38535m0000gp/T/ipykernel_23542/3814816774.py:20: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .agg(['mean', 'std'])


In [12]:
def bold_max_value(x, model_names):
    x = x.copy()
    len_cols = x.shape[0]
    n_models = len(model_names)
    idx_bold = (
        x
        [-n_models:]
        .str.replace(" ", "")
        .str.split("±")
        .str[0]
        .argmax()
    )
    x.iloc[idx_bold+len_cols-n_models] = '\\textbf{' + x.iloc[idx_bold+len_cols-n_models] + '}'
    
    return x


corruption_type_order = [
    'Original Data',
    'Chunk Shuffle',
    'Random Label',
    'Batch Label Shuffle',
    'Batch Label Flip',
    'Added Noise',
    'Replace With Noise',
]

corruption_type_map = {
    "no_c": "Original Data",
    "c_cs": "Chunk Shuffle",
    "c_rl": "Random Label",
    "c_lbs": "Batch Label Shuffle",
    "c_lbf": "Batch Label Flip",
    "c_ns": "Added Noise",
    "c_no": "Replace With Noise",
}

model_order = [
    "Standard",
    "IDPA",
    "Co-teaching",
    "LAP",
]


results_formatted = (
    results_all
    .loc[:, ('mean_std', 'test_top1acc')]
    .apply(
        bold_max_value,
        model_names = ["Co-teaching", "IDPA", "LAP", "Standard"],
        axis=1
    )
    .loc['cifar10']
    .rename_axis(index=None, columns=None)
    .reset_index()
    .rename(columns={"index": "Corruption Type"})
    .assign(
        **{
            "Corruption Type": lambda x: x['Corruption Type'].map(corruption_type_map),
        }
    )
    .set_index('Corruption Type')
    .loc[corruption_type_order, model_order]    
    .reset_index()
)

results_formatted

,Corruption Type,Standard,IDPA,Co-teaching,LAP
0,Original Data,69.46 ± 1.12,\textbf{70.86 ± 0.32},67.73 ± 0.83,69.6 ± 1.08
1,Chunk Shuffle,65.92 ± 1.32,66.34 ± 0.99,63.55 ± 0.41,\textbf{66.92 ± 0.56}
2,Random Label,60.85 ± 1.67,57.93 ± 1.59,61.37 ± 1.01,\textbf{66.5 ± 0.43}
3,Batch Label Shuffle,62.23 ± 0.62,60.08 ± 0.13,63.9 ± 0.44,\textbf{66.09 ± 0.65}
4,Batch Label Flip,59.94 ± 2.48,61.02 ± 1.9,63.05 ± 2.43,\textbf{64.48 ± 1.11}
5,Added Noise,59.86 ± 1.38,61.19 ± 0.33,59.59 ± 1.37,\textbf{63.58 ± 1.03}
6,Replace With Noise,65.07 ± 0.63,\textbf{65.85 ± 0.64},64.1 ± 1.13,64.89 ± 1.28


In [13]:
print(results_formatted.to_latex(index=False))

\begin{tabular}{lllll}
\toprule
Corruption Type & Standard & IDPA & Co-teaching & LAP \\
\midrule
Original Data & 69.46 ± 1.12 & \textbf{70.86 ± 0.32} & 67.73 ± 0.83 & 69.6 ± 1.08 \\
Chunk Shuffle & 65.92 ± 1.32 & 66.34 ± 0.99 & 63.55 ± 0.41 & \textbf{66.92 ± 0.56} \\
Random Label & 60.85 ± 1.67 & 57.93 ± 1.59 & 61.37 ± 1.01 & \textbf{66.5 ± 0.43} \\
Batch Label Shuffle & 62.23 ± 0.62 & 60.08 ± 0.13 & 63.9 ± 0.44 & \textbf{66.09 ± 0.65} \\
Batch Label Flip & 59.94 ± 2.48 & 61.02 ± 1.9 & 63.05 ± 2.43 & \textbf{64.48 ± 1.11} \\
Added Noise & 59.86 ± 1.38 & 61.19 ± 0.33 & 59.59 ± 1.37 & \textbf{63.58 ± 1.03} \\
Replace With Noise & 65.07 ± 0.63 & \textbf{65.85 ± 0.64} & 64.1 ± 1.13 & 64.89 ± 1.28 \\
\bottomrule
\end{tabular}



And the average accuracy over the last 10 epochs:

In [14]:
columns_intersection = [
    'dataset', 'corruption_type', 'run', 'epoch', 
    'method', 'test_top1acc', 'test_top5acc'
]

results_all = (
    pd.concat([
        results_df[columns_intersection],
        results_cot_df[columns_intersection],
        results_idpa_df[columns_intersection],
    ])
    .assign(test_top1acc=lambda x: x['test_top1acc']*100)
    .loc[lambda df: df["epoch"] > df["epoch"].max() - 10]
    .groupby(['dataset', 'corruption_type', 'method'])
    .agg({'test_top1acc': ['mean', 'std'], 'test_top5acc': ['mean', 'std']})
    .swaplevel(axis=1)
    .stack()
    .assign(
        mean_std = lambda x: 
            np.round(x['mean'], 2).astype(str) 
            + " ± "
            + np.round(x['std'], 2).astype(str),
    )
    ['mean_std']
    .to_frame()
    .reset_index()
    .rename({"level_3": "metric"}, axis=1)
    .loc[lambda df: df["metric"] == "test_top1acc"]
    .drop(columns="metric")
    .set_index(['dataset', 'corruption_type', 'method'])
    .unstack()
    .apply(
        bold_max_value,
        model_names = ["Co-teaching", "IDPA", "LAP", "Standard"],
        axis=1
    )
    ['mean_std']
    .loc['cifar10']
    .rename_axis(index=None, columns=None)
    .reset_index()
    .rename(columns={"index": "Corruption Type"})
    .assign(
        **{
            "Corruption Type": lambda x: x['Corruption Type'].map(corruption_type_map),
        }
    )
    .set_index('Corruption Type')
    .loc[corruption_type_order, model_order]    
    .reset_index()
)
results_all

,Corruption Type,Standard,IDPA,Co-teaching,LAP
0,Original Data,68.28 ± 1.4,\textbf{69.28 ± 0.71},66.35 ± 1.43,68.31 ± 1.34
1,Chunk Shuffle,64.66 ± 1.29,64.25 ± 1.0,62.13 ± 1.27,\textbf{65.59 ± 1.01}
2,Random Label,59.28 ± 1.77,51.43 ± 1.8,60.32 ± 1.29,\textbf{65.38 ± 0.87}
3,Batch Label Shuffle,60.89 ± 0.89,57.05 ± 1.15,63.1 ± 0.77,\textbf{64.94 ± 1.16}
4,Batch Label Flip,58.66 ± 2.5,57.73 ± 2.59,61.78 ± 2.63,\textbf{63.17 ± 1.75}
5,Added Noise,58.84 ± 1.26,60.03 ± 0.77,58.43 ± 1.45,\textbf{62.16 ± 1.44}
6,Replace With Noise,63.75 ± 1.19,\textbf{64.34 ± 0.67},62.79 ± 1.49,63.64 ± 1.31
